# Word2Vec Embedding

In [1]:
import pandas as pd

save_dir = ('../data/joongang_accident_token_df.csv')
df = pd.read_csv(save_dir)
df['tokenized'].head()

0    ['1일', '오후', '10', '시', '31', '분께', '대구', '달서구...
1    ['80대', '노모와', '지체', '장애', '를', '가진', '50대', '...
2    ['벌초를', '위해', '산에', '올랐던', '80대', '노인이', '실종',...
3    ['주말과', '휴일', '동안', '강원', '지역에서', '산악사고와', '교통...
4    ['제1', '2', '3', '석유', '류', '30', '만', '보관', '...
Name: tokenized, dtype: object

In [2]:
import ast

# list 형태를 하고있지만 실제로는 str이기 때문에 list로 변환
df['tokenized'] = df['tokenized'].map(lambda x: ast.literal_eval(x))

In [3]:
type(df['tokenized'][0])

list

In [4]:
result = [sentence for sentence in df['tokenized']]
for line in result[:3]:
  print(line)

['1일', '오후', '10', '시', '31', '분께', '대구', '달서구의', '한', '4층짜리', '모텔', '건물', '3층', '객실에', '서', '불이', '났다', '화재', '가', '발생', '하자', '건물', '안에', '있던', '손님', '30', '여명이', '급하게', '대피했', '으나', '불이', '시작된', '객실', '안에', '있던', '40대', '남성', 'A씨', '가', '연기를', '흡입', '해', '병원', '으로', '옮겨', '졌다', '또', '객실', '안', '침대', '등이', '불에', '탔다', '소방당국은', '소방', '차', '24대와', '소방', '관', '50', '명을', '투입해', '화재', '발생', '10', '여분', '뒤인', '오후', '10', '시', '45분께', '진화', '를', '완료했다', '대구소방본부', '관계자는', '화재', '원인', '등을', '조사', '하고', '있다', '고', '밝혔다', '연합뉴스']
['80대', '노모와', '지체', '장애', '를', '가진', '50대', '아들이', '집에서', '숨진', '채', '발견', '돼', '경찰', '이', '수사', '에', '착수했다', '1일', '서울', '강서경찰서에', '따르면', '이날', '오전', '4시께', '서울', '강서구', '의', '한', '아파트에서', '이', '집에', '사는', '80대', '여성', 'A씨', '와', '아들', '인', '50대', '남성', 'B씨', '가', '숨진', '채', '발견됐다', 'B씨', '는', 'A씨', '의', '큰아들로', '지체', '장애가', '있어', '평소', '거동이', '불편했던', '것으로', '알려졌다', '경찰', '관계자는', '모자의', '시신', '에서', '둔기', '에', '의한', '외상', '흔적', '이', '발견됐', '고', '타살', '혐의', '점이', '있다'

### Word2Vec 훈련시키기

하이퍼 파라미터
* size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
* window = 컨텍스트 윈도우 크기
* min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
* workers = 학습을 위한 프로세스 수
* sg = 0은 CBOW, 1은 Skip-gram.

In [7]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec): 
  """Callback to print loss after each epoch."""
  def __init__(self):
    self.epoch = 0
    self.loss_to_be_subed = 0
  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    loss_now = loss - self.loss_to_be_subed
    self.loss_to_be_subed = loss
    print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
    self.epoch += 1


model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=2, workers=4, sg=1, compute_loss=True, epochs=10, callbacks=[callback()])

Loss after epoch 0: 19702740.0
Loss after epoch 1: 14155888.0
Loss after epoch 2: 8729156.0
Loss after epoch 3: 8363200.0
Loss after epoch 4: 8264448.0
Loss after epoch 5: 7915144.0
Loss after epoch 6: 770344.0
Loss after epoch 7: 752736.0
Loss after epoch 8: 726984.0
Loss after epoch 9: 708224.0


### 수행 후 결과

In [8]:
model_result = model.wv.most_similar("폭행")
print(model_result)

[('마구', 0.7408427596092224), ('무차별', 0.7339503765106201), ('무자비하게', 0.7324828505516052), ('폭언과', 0.7286983728408813), ('상해', 0.7243751287460327), ('때린', 0.7230818271636963), ('폭언을', 0.7211732864379883), ('보복폭행', 0.7208795547485352), ('주먹과', 0.720650851726532), ('때려', 0.7081680297851562)]


### 모델 저장 및 로드

In [9]:
model.wv.save_word2vec_format('../model/w2v_01') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('../model/w2v_01') # 모델 로드

## Pre-Trained Model Load

In [1]:
from gensim.models import Word2Vec, KeyedVectors
pretrained_model = Word2Vec.load('../model/pre_trained/word2vec') # 모델 로드

In [2]:
model_result = pretrained_model.wv.most_similar("폭행")
print(model_result)

[('성추행', 0.8856818675994873), ('성폭행', 0.8785355687141418), ('구타', 0.8609321117401123), ('폭언', 0.8292363882064819), ('강간', 0.7983779907226562), ('협박', 0.7918735146522522), ('피해자', 0.7819958329200745), ('흉기', 0.7788309454917908), ('가해자', 0.7786863446235657), ('추행', 0.7670633792877197)]
